In [2]:
import torch
import torch.nn as nn 

In [41]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.downsample = downsample
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        identity = x.clone()
        
        x = self.bn1(self.conv1(x))
        x = self.relu(x)
        x = self.bn2(self.conv2(x))
        x = self.relu(x)
        
        if self.downsample is not None:
            x += self.downsample(identity)
        return x

In [48]:
class ResNet(nn.Module):
    def __init__(self, in_channels, num_classes, BasicBlock, layers):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU(inplace=True)
        
        self.layer1 = self._make_layer(BasicBlock, out_channels=64, num_rep=layers[0], stride=1)
        self.layer2 = self._make_layer(BasicBlock, out_channels=128, num_rep=layers[1], stride=2)
        self.layer3 = self._make_layer(BasicBlock, out_channels=256, num_rep=layers[2], stride=2)
        self.layer4 = self._make_layer(BasicBlock, out_channels=512, num_rep=layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=1)
        self.fc = nn.Linear(in_features=512, out_features=1000, bias=True)
        
    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = self.maxpool(self.relu(x))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x
    
    def _make_layer(self, BasicBlock, out_channels, num_rep, stride):
        layers = []
        
        downsample = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        
        if out_channels == 64:
            layers.append(BasicBlock(self.in_channels, out_channels))
        else:
            layers.append(BasicBlock(self.in_channels, out_channels, stride=2, downsample=downsample))
        self.in_channels = out_channels
        
        for i in range(num_rep-1):
            layers.append(BasicBlock(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)

### Written ResNet models using classes

In [50]:
ResNet18 = ResNet(in_channels=3, num_classes=1000, BasicBlock=BasicBlock, layers=[2, 2, 2, 2])
ResNet34 = ResNet(in_channels=3, num_classes=1000, BasicBlock=BasicBlock, layers=[3, 4, 6, 3])

### PyTorch pretrained ready to use ResNet models

In [49]:
model_18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model_34 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)

Using cache found in C:\Users\yerda/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\yerda/.cache\torch\hub\pytorch_vision_v0.10.0


### Let's load the pre-trained ResNet model's state weights to our models' 

In [51]:
ResNet18.load_state_dict(model_18.state_dict())

<All keys matched successfully>

In [52]:
ResNet34.load_state_dict(model_34.state_dict())

<All keys matched successfully>

In [53]:
x = torch.randn((4, 3, 224, 224))
ResNet18(x).shape

torch.Size([4, 1000])

In [54]:
model_18(x).shape

torch.Size([4, 1000])